## Deploy Notebook Function

by: Alex Comerford (alexanderjcomerford@gmail.com)

In this notebook we will guide the user through the steps to deploying a jupyter notebook as a serverless function on openfaas.

### Setup

First we will setup the openfaas cli and environment variables to allow us to directly talk to openfaas.

In [1]:
## Install the openfaas cli
!curl -sL https://cli.openfaas.com | sh

## Setup openfaas url for deployment    
%env OPENFAAS_URL=gateway-external.openfaas.svc.cluster.local:8080

sh: 1: ---: not found
sh: 2: redirect_from:: not found
sh: 3: ---: not found
x86_64
Download complete.

Running as root - Attempting to move faas-cli to /usr/local/bin
New version of faas-cli installed to /usr/local/bin
Creating alias 'faas' for 'faas-cli'.
  ___                   _____           ____
 / _ \ _ __   ___ _ __ |  ___|_ _  __ _/ ___|
| | | | '_ \ / _ \ '_ \| |_ / _` |/ _` \___ \
| |_| | |_) |  __/ | | |  _| (_| | (_| |___) |
 \___/| .__/ \___|_| |_|_|  \__,_|\__,_|____/
      |_|

CLI:
 commit:  a141dedf94ffeed84412365fd591bdc8999c5a1b
 version: 0.8.3
env: OPENFAAS_URL=gateway-external.openfaas.svc.cluster.local:8080


### Deploying a notebook

There are only two requirements to deploy a notebook as a function in openfaas

1. Write a jupyter notebook 
2. Deploy the 

In [210]:
NOTEBOOK_TO_DEPLOY = "./notebook_multiple_outputs.ipynb"

In [216]:
import os

funcname=os.path.splitext(
    os.path.basename(NOTEBOOK_TO_DEPLOY))[0].lower().replace("_","-")

!./faas-cli build --image={funcname} \
                  --handler=./       \
                  --name={funcname}  \
                  --lang=dockerfile  \
                  --no-cache         \
                  --build-arg notebook={NOTEBOOK_TO_DEPLOY} \
                  > /dev/null 2>&1

!./faas-cli deploy --image={funcname} \
                   --name={funcname}


Deployed. 202 Accepted.
URL: http://gateway-external.openfaas.svc.cluster.local:8080/function/notebook-multiple-outputs



## Test example stdout notebooks

In the next cell we will test (with curl) a notebook that outputs application/json to a file and output it to a file, then print that files json contents 

In [207]:
!curl -s -o /tmp/notebook-out-test \
      -H "Accept: application/json" \
      gateway-external.openfaas.svc.cluster.local:8080/function/notebook-out-test &&\
 cat /tmp/notebook-out-test

{'hello': 'i am a json!'}


## Test example mime-type output notebooks

In the next cell we will test (with curl) a notebook that outputs different mime-types to a file and output it to a file, then print that files json contents 

In [215]:
%%bash
curl -s -o /tmp/notebook-multiple-outputs \
     --data '{"TEST": "hello world","DATA": [9,9,9]}' \
     -H "Content-Type: application/json" \
     gateway-external.openfaas.svc.cluster.local:8080/function/notebook-multiple-outputs

cat /tmp/notebook-multiple-outputs